In [59]:
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import re
from PIL import Image,ImageOps

## Read Json files name

In [21]:
hate_data = [f for f in listdir('hateful_memes') if isfile(join('hateful_memes', f))]

In [22]:
hate_data

['dev_unseen.jsonl',
 'test_seen.jsonl',
 'dev_seen.jsonl',
 'train.jsonl',
 'README.md',
 'NLP_final_projetc_fb.ipynb',
 'LICENSE.txt',
 'test_unseen.jsonl']

### Select json file

In [23]:
josn_files_name = [i for i in hate_data if i.split('.')[1]=='jsonl']

In [24]:
josn_files_name_complete = ['hateful_memes/'+i for i in josn_files_name]
josn_files_name_complete

['hateful_memes/dev_unseen.jsonl',
 'hateful_memes/test_seen.jsonl',
 'hateful_memes/dev_seen.jsonl',
 'hateful_memes/train.jsonl',
 'hateful_memes/test_unseen.jsonl']

### read json file as dict

In [25]:
id_list = []
img_list = []
lable_list = []
text_list = []
wrong_line = []
for i in josn_files_name_complete:
    f = open(i).read()
    f_list = f.split('\n')
    for j in f_list:
        try:
            transfer_to_json = json.loads(str(j))
            id_list.append(transfer_to_json['id'])
            img_list.append('hateful_memes/'+transfer_to_json['img'])
            lable_list.append(transfer_to_json['label'])
            text_list.append(transfer_to_json['text'])
        except:
            wrong_line.append(j)

In [26]:
dataset = pd.DataFrame({'id':id_list, 'img':img_list, 'lable':lable_list, 'text':text_list})
dataset.head()

,id,img,lable,text
0,76432,hateful_memes/img/76432.png,0,you thinking what i'm thinking?
1,14270,hateful_memes/img/14270.png,0,a brilliant mind
2,56947,hateful_memes/img/56947.png,0,"pro gamer 6.000.000 kills, 1 death"
3,35174,hateful_memes/img/35174.png,0,lets end poaching
4,39264,hateful_memes/img/39264.png,0,my wife called me a good husband thats a huge ...


### Split into train and test

In [27]:
train = dataset.sample(frac=0.8, random_state=42)
test = dataset.drop(train.index)

In [28]:
train.head()

,id,img,lable,text
9526,61928,hateful_memes/img/61928.png,0,because of this guy i have multiple profile di...
2984,40217,hateful_memes/img/40217.png,0,2 years on the job... 4 million jobs gained 1 ...
6247,76582,hateful_memes/img/76582.png,0,how my friends introduce me she's our friend a...
9932,93245,hateful_memes/img/93245.png,0,when you tease your husband in public
7354,63418,hateful_memes/img/63418.png,0,damn! who am i gonna have to fuck to get this ...


In [29]:
test.head()

,id,img,lable,text
1,14270,hateful_memes/img/14270.png,0,a brilliant mind
4,39264,hateful_memes/img/39264.png,0,my wife called me a good husband thats a huge ...
5,18564,hateful_memes/img/18564.png,0,early america was cruel
9,51940,hateful_memes/img/51940.png,0,giving birth to a baby is one of the most memo...
11,30186,hateful_memes/img/30186.png,0,get you a man who will never lay a finger on y...


### let image size become same

In [67]:
def padding(img, expected_size):
    desired_size = expected_size
    delta_width = desired_size - img.size[0]
    delta_height = desired_size - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

In [68]:
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    # print(img.size)
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

In [98]:
train_label = list(train.lable)
test_label = list(test.lable)
train_img = list(train.img)
test_img = list(test.img)

0=not-hateful, 1=hateful

In [102]:
for i in range(len(train_img)):
    imag1 = Image.open(train_img[i])
    imag1 = resize_with_padding(imag1, (1000, 1000))
    if train_label[i]==1:
        imag1.save("fb_train/hate/"+(train_img[i].split('/'))[-1], 'PNG')
    else:
        imag1.save("fb_train/nothate/"+(train_img[i].split('/'))[-1], 'PNG')
        
for i in range(len(test_img)):
    imag1 = Image.open(test_img[i])
    imag1 = resize_with_padding(imag1, (1000, 1000))
    if test_label[i]==1:
        imag1.save("fb_test/hate/"+(test_img[i].split('/'))[-1], 'PNG')
    else:
        imag1.save("fb_test/nothate/"+(test_img[i].split('/'))[-1], 'PNG')

# 1. Image Classification

## Build model

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow import random
import numpy as np

In [2]:
seed = 1
np.random.seed(seed)
random.set_seed(seed)
classifier = Sequential()

Metal device set to: Apple M1


2021-12-20 23:57:37.199124: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-20 23:57:37.199463: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
classifier.add(Conv2D(128, 3, 3, input_shape = (1000, 1000, 3), activation = 'relu'))
classifier.add(Conv2D(64, 3, 3, activation = 'relu'))
classifier.add(Conv2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPool2D(2,2))
classifier.add(Conv2D(16, 3, 3, activation = 'relu'))
classifier.add(MaxPool2D(pool_size = (2,2)))
classifier.add(Flatten())
classifier.add(Dense(1024, activation='relu'))
classifier.add(Dense(512, activation='relu'))
classifier.add(Dense(256, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))

In [4]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
training_set = train_datagen.flow_from_directory('fb_train', target_size=(1000, 1000), batch_size=32, class_mode='binary')

Found 9766 images belonging to 3 classes.


In [8]:
test_set = train_datagen.flow_from_directory('fb_test', target_size=(1000, 1000), batch_size=32, class_mode='binary')

Found 2494 images belonging to 3 classes.


In [9]:
classifier.fit_generator(training_set, steps_per_epoch=100, epochs=20, validation_data=test_set, validation_steps=1000, shuffle=False)

/var/folders/0f/brs6kj9d6bbftjh0sk1763yh0000gn/T/ipykernel_87137/3259515234.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set, steps_per_epoch=100, epochs=20, validation_data=test_set, validation_steps=1000, shuffle=False)
2021-12-20 23:57:44.791192: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-20 23:57:44.979207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20
100/100 [==============================] - ETA: 0s - loss: -548788011008.0000 - accuracy: 0.3644

2021-12-21 00:03:25.774321: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


100/100 [==============================] - 603s 6s/step - loss: -548788011008.0000 - accuracy: 0.3644 - val_loss: -8382522589184.0000 - val_accuracy: 0.3825
Epoch 2/20
100/100 [==============================] - 337s 3s/step - loss: -11151900837675008.0000 - accuracy: 0.3547
Epoch 3/20
100/100 [==============================] - 336s 3s/step - loss: -2648064903847346176.0000 - accuracy: 0.3666
Epoch 4/20
100/100 [==============================] - 337s 3s/step - loss: -106488871030477553664.0000 - accuracy: 0.3691
Epoch 5/20
100/100 [==============================] - 337s 3s/step - loss: -1491605848121478742016.0000 - accuracy: 0.3709
Epoch 6/20
100/100 [==============================] - 339s 3s/step - loss: -9668619288114853052416.0000 - accuracy: 0.3675
Epoch 7/20
100/100 [==============================] - 338s 3s/step - loss: -42556589038452034502656.0000 - accuracy: 0.3719
Epoch 8/20
100/100 [==============================] - 338s 3s/step - loss: -133116154792319966511104.0000 - accur

## Sava and load model

In [13]:
classifier.save('saved_model/nlp_final_image')

2021-12-21 02:16:02.621660: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/nlp_final_image/assets


In [56]:
new_model = tf.keras.models.load_model('saved_model/nlp_final_image')

In [57]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 333, 333, 128)     3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 111, 111, 64)      73792     
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 37, 32)        18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 18, 18, 32)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 6, 16)          4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 16)         0         
 2D)                                                    

## Prediction

In [62]:
from keras.preprocessing import image

In [64]:
new_image = image.load_img('test/hate/098f8e5f97e5668547e70ad803e27923--macros-shake.jpg')

In [65]:
new_image = image.img_to_array(new_image)

In [70]:
new_imag1 = np.expand_dims(new_image, axis=0)

In [72]:
result = new_model.predict(new_imag1)

2021-12-21 03:37:24.989744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [73]:
if result[0][0] == 1:
    print(f'Is hate')
else:
    print(f'not hate')

Is hate


# 2. Text

In [31]:
train.to_csv('nlp_fb_train.csv')

In [32]:
test.to_csv('nlp_fb_test.csv')